# Introdução

<div style="text-align: justify"> O dataset contém características importantes para determinar um bom vinho, constituído por 1.599 observações e suas respectivas classificações sensoriais e psicoquímicas. Por questões de privacidade nenhum tipo de uva foi discriminada, assim como rótulos e preços. Temos disponíveis 12 variáveis distintas, sendo que 11 são características sensoriais/psicoquímicas e 1 com o score final do produto (escala de 0 - 10). </div>

Sumário de variáveis:

    1 - fixed acidity
    2 - volatile acidity
    3 - citric acid
    4 - residual sugar
    5 - chlorides
    6 - free sulfur dioxide
    7 - total sulfur dioxide
    8 - density
    9 - pH
    10 - sulphates
    11 - alcohol
    12 - quality (score between 0 and 10)

<div style="text-align: justify"> O objetivo é encontrar uma relação entre os componentes dos vinhos e suas respectivas notas, habilitando o comprador da importadora a ter uma pré-seleção com base no score atribuído. Entende-se como 'bom' vinho um score igual ou maior do que 6.

O dataset foi encontrado no repositório de machine learning da UCI: https://archive.ics.uci.edu/ml/datasets/wine+quality

Citação: P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009. </div>

# Bibliotecas

In [ ]:
import pydot
import numpy as np 
import pandas as pd
import seaborn as sns
from subprocess import call
from scipy.stats import zscore
from collections import Counter
from sklearn import preprocessing
from IPython.display import Image
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
%matplotlib inline

# Dataset

In [ ]:
dataset = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
dataset

In [ ]:
dataset.describe()

In [ ]:
dataset['quality'].value_counts()

In [ ]:
dataset.isnull().sum()

# Divisão das classes em duas

In [ ]:
dataset['good quality'] = [1 if x >= 6 else 0 for x in dataset['quality']]

In [ ]:
dataset.head()

In [ ]:
dataset['good quality'].value_counts()

# Matriz de correlação

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(data=dataset.corr(),annot=True, cmap="rocket")
plt.show()

# Box plot

In [ ]:
dataset.plot(kind='box',subplots=True,layout=(5,3),grid=True,figsize=(12,12))
plt.tight_layout()
plt.show()

# Remoção dos outliers

In [ ]:
filtered_entries = dataset[(np.abs(zscore(dataset)) < 3).all(axis=1)]
filtered_entries

# Train_test_split

In [ ]:
X = dataset.iloc[:,0:-2]
Y = dataset.iloc[:,-1]

In [ ]:
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X,Y,
                                        test_size=0.3,stratify=Y)

print('Quant de amostras de treino\n', Y_treino.value_counts())
print('Quant de amostras de teste\n', Y_teste.value_counts())

# Normalizar dataset

In [ ]:
scaler = preprocessing.MinMaxScaler().fit(X_treino)
X_treino_normalizado = scaler.transform(X_treino)
X_teste_normalizado = scaler.transform(X_teste)

# Random Forest Classifier

<div style="text-align: justify"> O funcionamento desse algoritmo se dá pela criação de muitas árvores de decisão, de maneira aleatória, formando o que podemos enxergar como uma floresta, onde cada árvore será utilizada na escolha do resultado final. </div>
<div style="text-align: justify"> As Árvores de Decisão, ou Decision Trees, estabelecem regras para tomada de decisão. O algoritmo criará uma estrutura similar a um fluxograma, com “nós” onde uma condição é verificada, e se atendida o fluxo segue por um ramo, caso contrário, por outro, sempre levando ao próximo nó, até a finalização da árvore. Com os dados de treino, o algoritmo busca as melhores condições, e onde inserir cada uma dentro do fluxo. </div>

    n_estimators - número de árvores
    max_depth - número máximo de profundidade da árvore
    min_samples_split - o número mínimo de amostras necessárias para dividir um nó interno
    random_state - controla a aleatoriedade da inicialização das amostras usadas ao construir árvores

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth = 4, min_samples_split = 0.1, random_state = 6)
rf.fit(X_treino_normalizado, Y_treino)

In [ ]:
len(rf.estimators_)

In [ ]:
estimator = rf.estimators_[5]

In [ ]:
export_graphviz(estimator, 
                out_file='tree.dot', 
                feature_names = X_treino.columns,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [ ]:
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

Image(filename = 'tree.png')

## Bar plot com importância das variáveis

In [ ]:
feat_importances = pd.Series(rf.feature_importances_, index=X.columns)
feat_importances.nlargest(25).plot(kind='barh',figsize=(8,8))

# Predições e cálculo da taxa de erro

In [ ]:
valores_preditos_teste = rf.predict(X_teste_normalizado)
valores_preditos_treinamento = rf.predict(X_treino_normalizado)

acuracia_teste = accuracy_score(Y_teste,
                                valores_preditos_teste)

acuracia_treinamento = accuracy_score(Y_treino,
                                      valores_preditos_treinamento)

kappa_teste = cohen_kappa_score(Y_teste,
                                      valores_preditos_teste)

kappa_treinamento = cohen_kappa_score(Y_treino,
                                      valores_preditos_treinamento)

matriz_confusao_teste = confusion_matrix(Y_teste,
                                         valores_preditos_teste)

print('Acuracia treino = ', acuracia_treinamento)
print('Acuracia teste = ', acuracia_teste)
print('Kappa treino = ', kappa_treinamento)
print('Kappa teste = ', kappa_teste)
print(matriz_confusao_teste)

# Procurando os melhores estimadores

    bootstrap - quando falso, todo o conjunto de dados é usado para construir cada árvore.

In [ ]:
parameters = {"max_depth" : np.linspace(10,100,10),"min_samples_leaf":[1,2,4],'min_samples_split':[2,5,10],'bootstrap':[True,False]}

In [ ]:
empty = RandomForestClassifier()

In [ ]:
Grid = GridSearchCV(empty,parameters,refit=True).fit(X_treino_normalizado, Y_treino)

In [ ]:
Grid.best_score_

In [ ]:
Grid.best_params_

In [ ]:
Grid.best_estimator_

In [ ]:
RFC = RandomForestClassifier(bootstrap = True, max_depth = 70.0, min_samples_leaf = 1, min_samples_split = 5).fit(X_treino_normalizado, Y_treino)

In [ ]:
valores_preditos_teste = RFC.predict(X_teste_normalizado)
valores_preditos_treinamento = RFC.predict(X_treino_normalizado)

acuracia_teste = accuracy_score(Y_teste,
                                valores_preditos_teste)

acuracia_treinamento = accuracy_score(Y_treino,
                                      valores_preditos_treinamento)

kappa_teste = cohen_kappa_score(Y_teste,
                                      valores_preditos_teste)

kappa_treinamento = cohen_kappa_score(Y_treino,
                                      valores_preditos_treinamento)

matriz_confusao_teste = confusion_matrix(Y_teste,
                                         valores_preditos_teste)

print('Acuracia treino = ', acuracia_treinamento)
print('Acuracia teste = ', acuracia_teste)
print('Kappa treino = ', kappa_treinamento)
print('Kappa teste = ', kappa_teste)
print(matriz_confusao_teste)

# Conclusões e Considerações

<div style="text-align: justify"> O dataset não possui nenhum null value;
Por outro lado, diversos outliers foram encontrados entre as variáveis, principalmente nas variáveis 'açucar resídual' e 'cloreto'; </div>

    'Acidez fixa': não é capaz de predizer a qualidade, uma vez que a variação entre diferentes notas foi parecida;
    'Acidez volátil': indica correlação entre indicador de acidez e nota. Quanto maior nota menor a acidez;
    'Acidez cítrica': não indica maior ou melhor qualidade, porém as notas mais altas (7 e 8) apontam maior equilíbrio entre as distribuições;
    'Açucar resídual': variação equilibrada mas com diversos outliers (especialmente com as notas intermediárias como 5 e 6);
    'Cloreto': variação equilibrada com diversos outliers (especialmente com as notas intermediárias como 5 e 6);
    'PH': notas mais altas (acima de 6) apresentaram limites inferiores menores do que as demais notas;
    'Sulfato': notas mais altas apresentaram maior índice de sulfato;
    'Álcool': notas mais altas apresentaram maior volume de álcool.

    
<div style="text-align: justify"> Ao analisar a importância das características para prever a qualidade, nos gráficos demostrou que o álcool, sulfatos, dióxido de enxofre total, densidade e acidez volátil têm um papel um pouco mais importante na previsão da classificação de qualidade do que as outras características. O método de ajuste dos parâmetros do estimador foram otimizados por uma pesquisa em grade com validação cruzada em uma grade de parâmetros, que foi a utilização do GridSearch no Random Forest que resultou no aumento da capacidade de classificação em quase 5 p.p. de acurácia no conjunto de teste e no kappa houve também um aumento de quase 9 p.p, mas infelizmente gerou Overfitting nos dados. </div>